# Modules To Import

In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json

# Crawler Class
> #### Note On Season Identifiers
> Season years are represented in th from of "Season year-year+1" (e.g. Season 2021-2022). To have less data complexity and more efficieny I choose "year+1" from each season in the form of "year-year+1" as a key for that specific season because the website [basketball-reference.com](www.basketball-reference.com) is also doing the same.
> 
>  **TL;DR**: The key "2024" points to the 2023-24 season.  

In [3]:
class Crawler:
    def get_mvps(self, mvp_page):
        mvp_soup = BeautifulSoup(mvp_page.content, "html.parser")
        mvp_soup = mvp_soup.find('table', id="mvp_NBA").tbody.find_all("tr")
        mvp_df = pd.DataFrame(columns=["player_name", "player_id", "team", "year"])

        # Scraps Required Elements from each row of the table
        for tr in mvp_soup:
            mvp_df.loc[len(mvp_df)] = [      
                tr.find(attrs={"data-stat":"player"}).a.string,
                tr.find(attrs={"data-stat":"player"}).a.get("href")[9:-5],
                tr.find(attrs={"data-stat":"team_id"}).a.get("href")[1:-5].split("/")[1],
                tr.find(attrs={"data-stat":"team_id"}).a.get("href")[1:-5].split("/")[-1]
            ]
        return mvp_df

    def get_player_data(self, player_series: pd.Series, player_page):
        player_series = player_series.copy()
#        player_page = requests.get("https://www.basketball-reference.com/players/" + player_series["player_id"] + ".html")
        player_soup = BeautifulSoup(player_page.content, "html.parser")
        return player_soup
        
        

In [4]:
mvp_page = requests.get("https://www.basketball-reference.com/awards/mvp.html")

In [26]:
c = Crawler()
mvp = c.get_mvps(mvp_page)
player_page = requests.get("https://www.basketball-reference.com/players/" + mvp.iloc[10]["player_id"] + ".html")

In [30]:
jokic = c.get_player_data(mvp.iloc[0], player_page)
jokic_json = json.loads(jokic.find("script", type="application/ld+json").string)
# experience
print(int(jokic.find("strong", string=re.compile(".*Experience:.*")).next_sibling.strip().split()[0]))
print(jokic.find("strong", string=re.compile(".*Shoots:.*")).next_sibling.strip())
print(jokic.find("span", string=re.compile(".*lb")).next_sibling.strip()[1:-1].split(",\xa0"))
print(jokic_json.get('birthPlace').split()[-1])
print(jokic_json.get('birthDate'))

position_string = jokic.find("strong", string=re.compile(".*Position:.*")).next_sibling
positions_cleaned = re.sub(r'[^\w\s,]', '', position_string)
position_list = re.split(r',|and', positions_cleaned)
position_list = [pos.strip() for pos in position_list if pos.strip()]
print(position_list)

16
Right
['211cm', '108kg']
States
1988-09-29
['Small Forward', 'Power Forward', 'Shooting Guard']
